In [1]:
import pandas as pd
import numpy as np
import re
from glob import glob
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

### Read in the data

In [2]:
path = 'house-office-expenditures-with-readme/'

In [3]:
# Data in 2017Q2-house-disburse-detail.csv is off by one column. Let's fix it
df_2017Q2_detail = pd.read_csv(path + '2017Q2-house-disburse-detail.csv', encoding='ISO-8859-1', header=0) 
df_2017Q2_detail.head()

,BIOGUIDE_ID,OFFICE,QUARTER,PROGRAM,CATEGORY,SORT SEQUENCE,DATE,TRANSCODE,RECORDID,PAYEE,START DATE,END DATE,PURPOSE,AMOUNT,YEAR
0,NaN,2017 OFFICE OF THE SPEAKER,2017Q2,GENERAL EXPENDITURES,PERSONNEL COMPENSATION,,,,ALTHOUSE JOSHUA S,4/1/17,6/30/17,CONSERVATIVE OUTREACH DIRECTOR,20000.01,2017,NaN
1,NaN,2017 OFFICE OF THE SPEAKER,2017Q2,GENERAL EXPENDITURES,PERSONNEL COMPENSATION,,,,ANDRES DOUGLAS R,4/1/17,6/30/17,PRESS SECRETARY,27500.01,2017,NaN
2,NaN,2017 OFFICE OF THE SPEAKER,2017Q2,GENERAL EXPENDITURES,PERSONNEL COMPENSATION,,,,ANDREWS THOMAS S,4/1/17,6/30/17,MEMBER SERVICES DIRECTOR,32500.00,2017,NaN
3,NaN,2017 OFFICE OF THE SPEAKER,2017Q2,GENERAL EXPENDITURES,PERSONNEL COMPENSATION,,,,ANTELL GEOFFREY,4/1/17,6/30/17,ASST TO THE SPEAKER FOR POLICY,41250.00,2017,NaN
4,NaN,2017 OFFICE OF THE SPEAKER,2017Q2,GENERAL EXPENDITURES,PERSONNEL COMPENSATION,,,,BENJAMIN WILLIAM C.,4/1/17,6/30/17,SYSTEM ADMINISTRATOR,13250.01,2017,NaN


In [4]:
# Shift data to the right columns
df_2017Q2_detail[df_2017Q2_detail.columns[-9:]] = df_2017Q2_detail[df_2017Q2_detail.columns[-10:-1].values]
df_2017Q2_detail[df_2017Q2_detail.columns[-10]] = ''

In [5]:
df_2017Q2_detail.head()

,BIOGUIDE_ID,OFFICE,QUARTER,PROGRAM,CATEGORY,SORT SEQUENCE,DATE,TRANSCODE,RECORDID,PAYEE,START DATE,END DATE,PURPOSE,AMOUNT,YEAR
0,NaN,2017 OFFICE OF THE SPEAKER,2017Q2,GENERAL EXPENDITURES,PERSONNEL COMPENSATION,,,,,ALTHOUSE JOSHUA S,4/1/17,6/30/17,CONSERVATIVE OUTREACH DIRECTOR,20000.01,2017
1,NaN,2017 OFFICE OF THE SPEAKER,2017Q2,GENERAL EXPENDITURES,PERSONNEL COMPENSATION,,,,,ANDRES DOUGLAS R,4/1/17,6/30/17,PRESS SECRETARY,27500.01,2017
2,NaN,2017 OFFICE OF THE SPEAKER,2017Q2,GENERAL EXPENDITURES,PERSONNEL COMPENSATION,,,,,ANDREWS THOMAS S,4/1/17,6/30/17,MEMBER SERVICES DIRECTOR,32500.00,2017
3,NaN,2017 OFFICE OF THE SPEAKER,2017Q2,GENERAL EXPENDITURES,PERSONNEL COMPENSATION,,,,,ANTELL GEOFFREY,4/1/17,6/30/17,ASST TO THE SPEAKER FOR POLICY,41250.00,2017
4,NaN,2017 OFFICE OF THE SPEAKER,2017Q2,GENERAL EXPENDITURES,PERSONNEL COMPENSATION,,,,,BENJAMIN WILLIAM C.,4/1/17,6/30/17,SYSTEM ADMINISTRATOR,13250.01,2017


In [6]:
df_2017Q2_detail['AMOUNT'].dtype

dtype('float64')

In [7]:
# Let's work on rest of the detail files
df = pd.DataFrame()
all_detail_files = glob(path + '*-detail.csv')
for file in [s for s in all_detail_files if '2017Q2' not in s]:
    detail_files = pd.read_csv(file, encoding='ISO-8859-1', header=0) 
    df = pd.concat([df, detail_files], ignore_index=True)

In [8]:
df.shape

(3435911, 17)

In [9]:
# Concatenate these detail files with the previous fixed 2017Q2 detail file
df = pd.concat([df, df_2017Q2_detail], ignore_index=True)
df.shape

(3542048, 17)

In [10]:
df.head()

,AMOUNT,BIOGUIDE_ID,CATEGORY,DATE,END DATE,OFFICE,PAYEE,PROGRAM,PURPOSE,QUARTER,RECIP (orig.),RECORDID,SORT SEQUENCE,START DATE,TRANSCODE,TRANSCODELONG,YEAR
0,455.00,NaN,OTHER SERVICES,NaN,03/02/10,COMMUNICATIONS,03Â­10 P2 MFP0003226 ...,NaN,NON-TECHNOLOGY SERVICE CONTRCT,2010Q1,03Â­10 P2 MFP0003226 ...,NaN,NaN,03/02/10,NaN,NaN,FISCAL YEAR 2010
1,47.26,NaN,SUPPLIES AND MATERIALS,NaN,11/28/09,COMMUNICATIONS,02Â­05 P2 MFP0003219 ALLSTEEL,NaN,HABITATION EXPENSES,2010Q1,02Â­05 P2 MFP0003219 ALLSTEEL,NaN,NaN,11/28/09,NaN,NaN,FISCAL YEAR 2010
2,250.00,NaN,SUPPLIES AND MATERIALS,NaN,12/21/09,COMMUNICATIONS,03Â­05 P2 OSM42304 CDW GOVERN...,NaN,OFFICE SUPPLIES OUTSIDE,2010Q1,03Â­05 P2 OSM42304 CDW GOVERN...,NaN,NaN,12/21/09,NaN,NaN,FISCAL YEAR 2010
3,436.00,NaN,SUPPLIES AND MATERIALS,NaN,12/21/09,COMMUNICATIONS,03Â­05 P2 OSM42304 DO,NaN,OFFICE SUPPLIES OUTSIDE,2010Q1,03Â­05 P2 OSM42304 DO,NaN,NaN,12/21/09,NaN,NaN,FISCAL YEAR 2010
4,37.90,NaN,SUPPLIES AND MATERIALS,NaN,12/21/09,COMMUNICATIONS,03Â­05 P2 OSM42304 DO,NaN,OFFICE SUPPLIES OUTSIDE,2010Q1,03Â­05 P2 OSM42304 DO,NaN,NaN,12/21/09,NaN,NaN,FISCAL YEAR 2010


In [11]:
df.dtypes

AMOUNT           object
BIOGUIDE_ID      object
CATEGORY         object
DATE             object
END DATE         object
OFFICE           object
PAYEE            object
PROGRAM          object
PURPOSE          object
QUARTER          object
RECIP (orig.)    object
RECORDID         object
SORT SEQUENCE    object
START DATE       object
TRANSCODE        object
TRANSCODELONG    object
YEAR             object
dtype: object

### Function to convert AMOUNT from string to numeric

In [12]:
def convert_currency(val):
    # Convert the string number value to a float by removing commas, then convert to float type
    new_val = str(val)
    return float(new_val.replace(',',''))

In [13]:
df['AMOUNT'] = df['AMOUNT'].apply(convert_currency)
df.dtypes

AMOUNT           float64
BIOGUIDE_ID       object
CATEGORY          object
DATE              object
END DATE          object
OFFICE            object
PAYEE             object
PROGRAM           object
PURPOSE           object
QUARTER           object
RECIP (orig.)     object
RECORDID          object
SORT SEQUENCE     object
START DATE        object
TRANSCODE         object
TRANSCODELONG     object
YEAR              object
dtype: object

### Save the cleaned data file

In [14]:
df.to_csv('detail_files.csv', index=False)

### Read in the new cleaned data file

In [15]:
df = pd.read_csv('detail_files.csv')

### Question 1: What is the total of all the payments in the dataset?

In [16]:
total_payment = df['AMOUNT'].sum()
print(f'Question 1: {total_payment}')

Question 1: 13660703793.310003


### Question 2: Define the 'COVERAGE PERIOD' for each payment as the difference (in days) between 'END DATE' and 'START DATE'. What is the standard deviation in 'COVERAGE PERIOD'? Only consider payments with strictly positive amounts.

In [17]:
# Convert START DATE and END DATE from string to datetime type
df['START DATE'] = pd.to_datetime(df['START DATE'], format='%m/%d/%y', errors = 'coerce')
df['END DATE'] = pd.to_datetime(df['END DATE'], format='%m/%d/%y', errors = 'coerce')
df.dtypes

AMOUNT                  float64
BIOGUIDE_ID              object
CATEGORY                 object
DATE                     object
END DATE         datetime64[ns]
OFFICE                   object
PAYEE                    object
PROGRAM                  object
PURPOSE                  object
QUARTER                  object
RECIP (orig.)            object
RECORDID                 object
SORT SEQUENCE            object
START DATE       datetime64[ns]
TRANSCODE                object
TRANSCODELONG            object
YEAR                     object
dtype: object

In [18]:
df['COVERAGE PERIOD'] = df['END DATE'] - df['START DATE']
df['COVERAGE PERIOD']

0          0 days
1          0 days
2          0 days
3          0 days
4          0 days
5         30 days
6         27 days
7          0 days
8          0 days
9          6 days
10         6 days
11         0 days
12         0 days
13        30 days
14         9 days
15         9 days
16         9 days
17         9 days
18         9 days
19         9 days
20         9 days
21         9 days
22        23 days
23        23 days
24        23 days
25        23 days
26        23 days
27        23 days
28        23 days
29        23 days
            ...  
3542018       NaT
3542019       NaT
3542020       NaT
3542021   27 days
3542022   29 days
3542023   29 days
3542024   30 days
3542025   30 days
3542026   29 days
3542027   29 days
3542028   30 days
3542029   56 days
3542030   30 days
3542031   29 days
3542032   29 days
3542033   60 days
3542034   29 days
3542035   29 days
3542036   30 days
3542037   29 days
3542038   30 days
3542039   30 days
3542040       NaT
3542041       NaT
3542042   

In [19]:
std = df[df['AMOUNT']>0]['COVERAGE PERIOD'].std().total_seconds()/60/60/24
print(f'Question 2: {std:.10f}')

Question 2: 61.8808082569


### Question 3: What was the average annual expenditure with a 'START DATE' date between January 1, 2010 and December 31, 2016 (inclusive)? Only consider payments with strictly positive amounts.

In [20]:
df_2010_2016 =  df[(df['START DATE']>='2010-01-01') & (df['START DATE']<='2016-12-31') & (df['AMOUNT']>0)]
df_2010_2016.shape

(2692214, 18)

In [21]:
total = df_2010_2016['AMOUNT'].sum()/7
print(f'Question 3: {total}')

Question 3: 1209984621.0528576


### Question 4: Find the 'OFFICE' with the highest total expenditures with a 'START DATE' in 2016. For this office, find the 'PURPOSE' that accounts for the highest total expenditures. What fraction of the total expenditures (all records, all offices) with a 'START DATE' in 2016 do these expenditures amount to? 

In [22]:
df_2016 =  df[(df['START DATE']>='2016-01-01') & (df['START DATE']<='2016-12-31')]
df_2016.head()

,AMOUNT,BIOGUIDE_ID,CATEGORY,DATE,END DATE,OFFICE,PAYEE,PROGRAM,PURPOSE,QUARTER,RECIP (orig.),RECORDID,SORT SEQUENCE,START DATE,TRANSCODE,TRANSCODELONG,YEAR,COVERAGE PERIOD
459592,79.0,F000460,SUPPLIES AND MATERIALS,01-05,2017-01-08,HON. BLAKE FARENTHOLD,ROCKPORT PILOT,NaN,PUBLICATIONS/REFERENCE MAT'L,2015Q1,ROCKPORT PILOT,E0231342,NaN,2016-01-09,AP,Accounts payable,2014,365 days
466444,3250.0,G000569,SUPPLIES AND MATERIALS,03-31,2016-12-31,HON. MICHAEL G. GRIMM,ARISTOTLE INTERNATIONAL INC,NaN,PUBLICATIONS/REFERENCE MAT'L,2015Q1,ARISTOTLE INTERNATIONAL INC,E0262887,NaN,2016-01-01,AP,Accounts payable,2015,365 days
480980,9998.0,L000554,SUPPLIES AND MATERIALS,01-09,2016-12-31,HON. FRANK A. LOBIONDO,MORNINGSIDE PARTNERS LLC,NaN,PUBLICATIONS/REFERENCE MAT'L,2015Q1,MORNINGSIDE PARTNERS LLC,00768917,NaN,2016-01-01,AP,Accounts payable,2014,365 days
490368,5850.0,N000179,OTHER SERVICES,01-05,2016-12-31,HON. GRACE F. NAPOLITANO,BLOOMBERG LP,NaN,NON-TECHNOLOGY SERVICE CONTR,2015Q1,BLOOMBERG LP,E0232243,NaN,2016-01-01,AP,Accounts payable,2014,365 days
492619,780.0,O000170,SUPPLIES AND MATERIALS,01-05,2016-12-31,HON. BETO OROURKE,"WYSONG, DAVID M",NaN,PUBLICATIONS/REFERENCE MAT'L,2015Q1,"WYSONG, DAVID M",E0232032,NaN,2016-01-01,AP,Accounts payable,2014,365 days


In [23]:
df_2016.groupby(['OFFICE'])[['AMOUNT']].sum().sort_values(by='AMOUNT', ascending=False).head()

,AMOUNT
OFFICE,
GOVERNMENT CONTRIBUTIONS,1.664793e+08
CHIEF ADMIN OFCR OF THE HOUSE,1.199160e+08
COMMITTEE ON APPROPRIATIONS,2.516048e+07
FISCAL YEAR 2017 GOVERNMENT CONTRIBUTIONS,2.330127e+07
CLERK OF THE HOUSE,2.291365e+07


In [24]:
highest_total_office = df_2016[df_2016['OFFICE']=='GOVERNMENT CONTRIBUTIONS']
highest_total_office.head()

,AMOUNT,BIOGUIDE_ID,CATEGORY,DATE,END DATE,OFFICE,PAYEE,PROGRAM,PURPOSE,QUARTER,RECIP (orig.),RECORDID,SORT SEQUENCE,START DATE,TRANSCODE,TRANSCODELONG,YEAR,COVERAGE PERIOD
1198188,2752.04,NaN,PERSONNEL COMPENSATION,NaN,2016-04-30,GOVERNMENT CONTRIBUTIONS,"STARKEY,CHARLES J",NaN,BUSINESS CONTINUITY MANAGER,2016Q2,"STARKEY,CHARLES J",NaN,NaN,2016-04-01,NaN,NaN,FISCAL YEAR 2016,29 days
1198189,811.92,NaN,PERSONNEL COMPENSATION,NaN,2016-04-30,GOVERNMENT CONTRIBUTIONS,"WRIGHT,ASLI V",NaN,SENIOR CONTRACTS SPECIALIST,2016Q2,"WRIGHT,ASLI V",NaN,NaN,2016-04-01,NaN,NaN,FISCAL YEAR 2016,29 days
1198191,7062.41,NaN,PERSONNEL COMPENSATION,NaN,2016-06-30,GOVERNMENT CONTRIBUTIONS,"STARKEY,CHARLES J",NaN,BUSINESS CONTINUITY MANAGER,2016Q2,"STARKEY,CHARLES J",NaN,NaN,2016-05-01,NaN,NaN,FISCAL YEAR 2016,60 days
1198192,1834.59,NaN,PERSONNEL COMPENSATION,NaN,2016-06-30,GOVERNMENT CONTRIBUTIONS,"WRIGHT,ASLI V",NaN,SENIOR CONTRACTS SPECIALIST,2016Q2,"WRIGHT,ASLI V",NaN,NaN,2016-05-01,NaN,NaN,FISCAL YEAR 2016,60 days
1198193,21.50,NaN,PERSONNEL BENEFITS,04-01,2016-02-29,GOVERNMENT CONTRIBUTIONS,"SOLLAZZO, AMANDA M",NaN,TRANSIT BENEFITS,2016Q2,"SOLLAZZO, AMANDA M",E0386764,NaN,2016-02-23,AP,Accounts payable,FISCAL YEAR 2016,6 days


In [25]:
highest_total_office.groupby(['PURPOSE'])[['AMOUNT']].sum().sort_values(by='AMOUNT', ascending=False).head()

,AMOUNT
PURPOSE,
FERS,62452380.59
HEALTH INSURANCE,26565188.31
STUDENT LOANS,14661130.44
TSP MATCHING,13976878.79
FURTHER FERS RAE,11598185.14


In [26]:
fraction = df_2016[df_2016['PURPOSE']=='FERS']['AMOUNT'].sum() / df_2016['AMOUNT'].sum()
print(f'Question 4: {fraction:.10f}')

Question 4: 0.0568838384


### Question 5: What was the highest average staff salary among all representatives in 2016? Assume staff sizes is equal to the number of unique payees in the 'PERSONNEL COMPENSATION' category for each representative.

In [31]:
df_staff_2016 = df_2016[(df_2016['CATEGORY']=='PERSONNEL COMPENSATION') & (df_2016['BIOGUIDE_ID'].notnull())]
df_staff_2016.head()

,AMOUNT,BIOGUIDE_ID,CATEGORY,DATE,END DATE,OFFICE,PAYEE,PROGRAM,PURPOSE,QUARTER,RECIP (orig.),RECORDID,SORT SEQUENCE,START DATE,TRANSCODE,TRANSCODELONG,YEAR,COVERAGE PERIOD
1114547,9687.50,A000374,PERSONNEL COMPENSATION,NaN,2016-06-30,HON. RALPH ABRAHAM,"ARNOLD,EMILY M",NaN,CASEWORKER,2016Q2,"ARNOLD,EMILY M",NaN,NaN,2016-04-01,NaN,NaN,2016,90 days
1114548,17062.50,A000374,PERSONNEL COMPENSATION,NaN,2016-06-30,HON. RALPH ABRAHAM,"AVERY,ROBERT C",NaN,COMMUNICATIONS DIRECTOR,2016Q2,"AVERY,ROBERT C",NaN,NaN,2016-04-01,NaN,NaN,2016,90 days
1114549,12999.99,A000374,PERSONNEL COMPENSATION,NaN,2016-06-30,HON. RALPH ABRAHAM,"BARRON,PATRICK C",NaN,LEGISLATIVE ASSISTANT,2016Q2,"BARRON,PATRICK C",NaN,NaN,2016-04-01,NaN,NaN,2016,90 days
1114550,11000.00,A000374,PERSONNEL COMPENSATION,NaN,2016-06-30,HON. RALPH ABRAHAM,"BOIES,LILIA C",NaN,OFFICE MANAGER,2016Q2,"BOIES,LILIA C",NaN,NaN,2016-04-01,NaN,NaN,2016,90 days
1114551,14937.50,A000374,PERSONNEL COMPENSATION,NaN,2016-06-30,HON. RALPH ABRAHAM,"BROWN,ALAN K",NaN,DIRECTOR OF DISTRICT OUTREACH,2016Q2,"BROWN,ALAN K",NaN,NaN,2016-04-01,NaN,NaN,2016,90 days


In [38]:
# Clean data in PAYEE
df_staff_2016['PAYEE'] = df_staff_2016['PAYEE'].astype(str).map(lambda x: x.replace(',','').replace('.','').replace(' ',''))

In [39]:
groupby_staff = pd.DataFrame({'AMOUNT_SUM' : df_staff_2016.groupby(['BIOGUIDE_ID', 'PAYEE'])['AMOUNT'].sum()}).reset_index()

In [41]:
groupby_staff.head()

,BIOGUIDE_ID,PAYEE,AMOUNT_SUM
0,A000055,ABERNATHYPAMELAM,57131.95
1,A000055,CHRISTENSENAUTUMN,6500.00
2,A000055,CLARKCARSONG,59202.74
3,A000055,DAWSONMARKE,11966.67
4,A000055,DONCHESMICHELLEM,15999.96


In [43]:
highest_average_salary = pd.DataFrame(groupby_staff.groupby(['BIOGUIDE_ID'])['AMOUNT_SUM'].mean()).sort_values('AMOUNT_SUM', ascending = False)['AMOUNT_SUM'].values[0]
print(f'Question 5: {highest_average_salary:.10f}')

Question 5: 65064.5381250000


### Question 6: What was the median rate of annual turnover in staff between 2011 and 2016 (inclusive)? Turnover for 2011 should be calculated as the fraction of a representative's staff from 2010 who did not carry over to 2011. Only consider representatives who served for at least 4 years and had staff size of at least 5 every year that they served.

### Question 7: What percentage of the expenditures of the top 20 spenders in 2016 come from members of the Democratic Party? Representatives are identified by their 'BIOGUIDE_ID', which can be used to look up representatives with ProPublica's Congress API to find their party affiliation. Consider an expenditure as being in 2016 if its 'START DATE' is in 2016.